In [15]:
import requests

In [33]:
r = requests.get(r"https://www.sephora.com/product/P19453367")

In [34]:
result = re.findall(r"s\d+?\+sw\.jpg", r.text)

In [35]:
len(result), len(set(result))

(29, 9)

---

In [4]:
with open("listing.txt") as fin:
    t = fin.read()

In [5]:
import re

In [6]:
pids = set(re.findall(r"(?:grid:)(.*?)(?::)", t))
pids.add("P443336")

In [7]:
len(pids)

300

---

In [48]:
list(pids)[0]

'p19453367'

In [49]:
import time
import random

In [50]:
TEMPLATE_URL = r"https://www.sephora.com/product/{pid}"

In [67]:
# result = {}
# for i, pid in enumerate(pids):
#     if i % 10 == 0:
#         print(i)
#     url = TEMPLATE_URL.format(pid=pid.upper())
#     r = requests.get(url)
#     skus = set(re.findall(r"s\d+?\+sw\.jpg", r.text))
#     result[pid] = skus
#     time.sleep(0.5 + random.random())
# #     if i == 2:
# #         break

In [68]:
# [(pid, len(skus)) for pid, skus in result.items()]

In [69]:
# sum(len(skus) for skus in result.values())

In [70]:
# sku2desc = {}

In [71]:
# for pid, skus in result.items():
#     for sku in skus:
#         sku = sku[1:][:-7]
#         sku2desc[sku] = {
#             "pid" : pid,
#         }

In [72]:
# sku2desc

save file

In [60]:
for i, pid in enumerate(pids):
    if i % 30 == 0:
        print(i)
    url = TEMPLATE_URL.format(pid=pid.upper())
    r = requests.get(url)
    with open(f"product page/{pid}.html", "w", encoding="utf-8") as fout:
        fout.write(r.text)

0
30
60
90
120
150
180
210
240
270


parse

In [9]:
import os
import re

In [10]:
from bs4 import BeautifulSoup

In [11]:
def get_brand_and_name(soup, debug=False):
    display_name = soup.find_all("h1", {"data-comp":"DisplayName Flex Box"})[0]
    brand = display_name.findChildren("a" , recursive=False)[0]\
            .findChildren("span" , recursive=False)[0].text
    name  = display_name.findChildren("span" , recursive=False)[0].text
    if debug:
        print(brand, name)
    return brand, name

In [12]:
def get_sku2code_list(soup, debug=False):
    sku2code = {}
    buttons = soup.find_all("button", {"data-at":"selected_swatch"})
    for button in buttons:
        code = button.get("aria-label")
        src = str(button)
        skus = re.findall(r's\d+?\+sw',src)
        for sku in skus:
            sku = sku[1:][:-3]
            sku2code[sku] = code
        if debug:
            print(code)
            print(src)
            print(skus)
    return sku2code

In [19]:
def get_price_str(soup, debug=False):
    box = soup.find("div", {"data-comp":"Price Box"})
    if not box:
        return ""
    children = box.findChildren()
    if len(children) > 1:
        child = children[0]
        return child.text
    else:
        return box.text


In [22]:
result = {}

for root, dirs, files in os.walk("product page"):
    tot = len(files)
    for i, file in enumerate(files):
        if i % 50 == 0:
            print(i)
        full_path = os.path.join("product page", file)
        size = os.stat(full_path).st_size
        if size < 150_000:
            continue
        pid = file[:-5].upper()
        with open(full_path, "r", encoding="utf-8") as fin:
            t = fin.read()
        soup = BeautifulSoup(t)
        brand, name = get_brand_and_name(soup)
        sku2code = get_sku2code_list(soup)
        price_str = get_price_str(soup)
        result[pid] = {
            "brand" : brand,
            "name"  : name,
            "sku2code" : sku2code,
            "price" : price_str,
        }
#         break

0
50
100
150
200
250


In [23]:
result

{'P09784333': {'brand': 'FENTY BEAUTY by Rihanna',
  'name': 'Snow Daze & Snow Nights Frosted Metal Lipstick 3-pc Set',
  'sku2code': {'2144616': 'Snow Nights No Chill, Frost Sauce, Fog Snog - Selected',
   '2144608': "Snow Daze Cuffing Season, What's Your Numb'a, So Chilli"},
  'price': '$36.00'},
 'P10246544': {'brand': 'NARS',
  'name': 'Full Vinyl Lip Lacquer',
  'sku2code': {'2063766': 'Orgasm peachy pink w/ golden shimmer - Selected',
   '2090025': 'Cape Town metallic gold with pink shimmer',
   '2089993': 'Red District vivid red',
   '2089985': 'Santo Domingo mauve pink',
   '2090017': 'Abruzzo metallic rose gold',
   '2090009': 'Mississippi dark black cherry'},
  'price': '$26.00'},
 'P122751': {'brand': 'CLINIQUE',
  'name': 'Almost Lipstick',
  'sku2code': {'70680': 'Black Honey', '1719251': 'Pink Honey - Selected'},
  'price': '$18.50'},
 'P181912': {'brand': 'Givenchy',
  'name': 'Rouge Interdit Satin Lipstick',
  'sku2code': {'1897735': '01 Secret Nude natural beige',
   '

use sku to index desc

```json

{ 
    sku : {
        "pid": pid,  # capitalized "P"
        "brand": brand,
        "name": name,
        "code": code
        "price": price
    }
}
```

In [25]:
sku2desc = {}

for pid, buf in result.items():
    name = buf["name"]
    brand = buf["brand"]
    price = buf["price"]
    for sku, code in buf["sku2code"].items():
        sku2desc[sku] = {
            "pid"   : pid,
            "brand" : brand,
            "name"  : name,
            "code"  : code,
            "price" : price,
        }

In [26]:
sku2desc

{'2144616': {'pid': 'P09784333',
  'brand': 'FENTY BEAUTY by Rihanna',
  'name': 'Snow Daze & Snow Nights Frosted Metal Lipstick 3-pc Set',
  'code': 'Snow Nights No Chill, Frost Sauce, Fog Snog - Selected',
  'price': '$36.00'},
 '2144608': {'pid': 'P09784333',
  'brand': 'FENTY BEAUTY by Rihanna',
  'name': 'Snow Daze & Snow Nights Frosted Metal Lipstick 3-pc Set',
  'code': "Snow Daze Cuffing Season, What's Your Numb'a, So Chilli",
  'price': '$36.00'},
 '2063766': {'pid': 'P10246544',
  'brand': 'NARS',
  'name': 'Full Vinyl Lip Lacquer',
  'code': 'Orgasm peachy pink w/ golden shimmer - Selected',
  'price': '$26.00'},
 '2090025': {'pid': 'P10246544',
  'brand': 'NARS',
  'name': 'Full Vinyl Lip Lacquer',
  'code': 'Cape Town metallic gold with pink shimmer',
  'price': '$26.00'},
 '2089993': {'pid': 'P10246544',
  'brand': 'NARS',
  'name': 'Full Vinyl Lip Lacquer',
  'code': 'Red District vivid red',
  'price': '$26.00'},
 '2089985': {'pid': 'P10246544',
  'brand': 'NARS',
  'na

In [27]:
import json

In [28]:
with open("sku2desc.json", "w", encoding="utf-8") as fout:
    json.dump(sku2desc, fout)